![banner](https://github.com/paulguz261/MIAD_20242_proyecto_despliegue_aplicaciones/blob/main/docs/images/despliegue.png)

# Importación de librerías  

In [27]:
import pandas as pd
import numpy as np
from IPython.display import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

# Cargar datos

In [2]:
data = pd.read_csv("../data/clean/personas_limpio.csv")
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.600000,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.893237,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.500000,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.400000,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.000000,never smoked,1


In [19]:
# Dado que las variables binarias quedaron como númericas, se procede a convertirlas de nuevo
def convert_to_categorical(df, columns):
    """
    Convierte las columnas especificadas de un DataFrame a tipo categórico.

    Parámetros:
    df (pd.DataFrame): El DataFrame en el que se realizarán las conversiones.
    columns (list): Lista de nombres de columnas a convertir a tipo categórico.

    Retorna:
    pd.DataFrame: El DataFrame con las columnas especificadas convertidas a tipo categórico.
    """
    for column in columns:
        if column in df.columns:
            df[column] = pd.Categorical(df[column])
        else:
            print(f"Columna '{column}' no encontrada en el DataFrame.")
    return df

categorical_columns = ['hypertension', 'heart_disease','stroke']
data = convert_to_categorical(data, categorical_columns)

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   gender             5110 non-null   object  
 1   age                5110 non-null   float64 
 2   hypertension       5110 non-null   category
 3   heart_disease      5110 non-null   category
 4   ever_married       5110 non-null   object  
 5   work_type          5110 non-null   object  
 6   Residence_type     5110 non-null   object  
 7   avg_glucose_level  5110 non-null   float64 
 8   bmi                5110 non-null   float64 
 9   smoking_status     5110 non-null   object  
 10  stroke             5110 non-null   category
dtypes: category(3), float64(3), object(5)
memory usage: 334.8+ KB


El conjunto de datos `personas_limpio`contiene 5.110 registros y 11 columnas resultado de la limpieza de datos aplicada. Las variables categóricas son `gender`, `ever_married`, `work_type`, `resident_type`y `smoking_status`, variables que indican factores importantes a la hora de predecir un ACV. Además, las variables númericas tales `age`, `avg_glucose_level` y `bmi` indican edad y estado de salud de la persona. Y las variables binarias `hypertension` y `heart_disase`, las cuales indican el padecimiento de cierta patologia. Finalmente, la variable a predecir `stroke` indica la presencia de ACV en la persona.

# Selección de variables

In [22]:
# Conversión de variables categóricas a numéricas con LabelEncoder
for column in data.select_dtypes(include=['object','category']).columns:
    data[column] = LabelEncoder().fit_transform(data[column])

# Definición de las variables predictoras y a predecir
X = data.drop(columns=['stroke'])  # Eliminar variable a predecir
y = data['stroke']

# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=40)

# XGBoost

In [28]:
clf = XGBClassifier()
# Entrenamiento (fit) y desempeño del modelo XGBClassifier
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.f1_score(y_pred, y_test.values), metrics.accuracy_score(y_pred, y_test.values)

(0.12371134020618557, 0.949614700652045)